# Extract Weights and Visualize the Activations

In [ ]:
import time
import torch
import itertools
import torchvision

import numpy as np
import torch.nn.functional as Functional
import matplotlib.pyplot as plt

from torch import nn, optim
from torchvision import datasets, transforms

from torch.utils.data import DataLoader

In [ ]:
torch.manual_seed(13)

N_train = 64
N_test = 256

# We will use torch.utils.data.DataLoader to wrap our dataset.
# This provides easier batching, GPU support, etc.
# Calling torchvision.datasets.MNIST() will download and format the MNIST
# dataset with the transforms we specify. Here, in the transforms we first convert
# the image to PyTorch tensor, and then normalize the image based on a given mean
# and standard deviation. Normalizing the image does: image = (image - mean) / std.
# We shuffle the data as well by defining shuffle=True.

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../Datasets/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=N_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../Datasets/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=N_test, shuffle=False)

In [ ]:
test_subset = enumerate(test_loader)
batch_idx, (one_batch_of_test_subset_x, one_batch_of_test_subset_y) = next(test_subset)

In [ ]:
i = 0
plt.imshow(one_batch_of_test_subset_x[i][0], cmap='gray', interpolation='none')
_ = plt.title("Ground Truth: {}".format(one_batch_of_test_subset_y[i]))
number7 = one_batch_of_test_subset_x[i][0]

In [ ]:
class CNN_A(nn.Module):
    def __init__(self):
        super(CNN_A, self).__init__()
        # We can define the arguments of each layer in the __init__ method.
        # __init__ method will be called everytime we create an object of this class.
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(1600, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        # This is the forward pass function.
        # See how we can save the activation outputs of each layer into a variable.
        # In this case, we are saving the output of each layer
        # to the same variable and replacing the value every time
        # before sending to a new layers.
        
        # Conv -> MaxPool -> ReLU
        x = self.conv1(x)
        x = Functional.max_pool2d(x, 2)
        x = Functional.relu(x)
        
        # Conv -> MaxPool -> ReLU -> Dropout -> Flatten
        x = self.conv2(x)
        x = Functional.max_pool2d(x, 2)
        x = Functional.relu(x)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        
        # 3-layer MLP
        x = self.fc1(x)
        x = Functional.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = Functional.relu(x)      
        
        return x

In [ ]:
model = torch.load('cnn_a_model.pt')

In [ ]:
model

# Analysis of Weight Matrices/Parameters

In [ ]:
def model_summary(model):
    """
    Returns the number of parameters (trainable and total) of a PyTorch model.
    """
    print("Trainable parameter variables: {}\nTotal number of parameters: {}\nTotal number of trainable parameters: {}".format(
        len(list(model.parameters())),
        sum(p.numel() for p in model.parameters()),
        sum(p.numel() for p in model.parameters() if p.requires_grad))
         )

In [ ]:
model_summary(model)

In [ ]:
# Extract the weights of the kernels of conv1 layer.
# We can later see how the weights vary per each layer.
kernels_conv1 = model.conv1.weight.cpu().detach().clone().numpy()

In [ ]:
kernels_conv1.shape

In [ ]:
for _, i in enumerate(kernels_conv1):
    plt.imshow(i[0], cmap='gray')
    plt.show()
    if _==5:
        break

# Activation Maps

Similar to how we studied the embeddings in DNNs, in CNNs we are also interested in what kernels/filters the CNN had learnt during the optimization. To do so, we can do a forward pass by providing an input to the model and see what each kernels provide as an output. By visualizing these outputs, or activations, we can study (visually) what the network is learning. 

### Extract Conv1 layer activations

$$\frac{W-K+2P}{S} +1$$

- W is the input volume
- K is the kernel size
- P is the amount of padding
- S is the stride size

In [ ]:
# Do a forward pass on the first convolution layer by passing
# the original MNIST image data to it.

conv1_activations = model.conv1.forward(number7.reshape(1,1,28,28)) # NCHW

# Alternate way to extract activations.
# conv1_layer = nn.Sequential(*list(model.children()))[0]
# conv1_activations = conv1_layer(number7.reshape(1,1,28,28))

Conv2 activations can be extracted by doing a forward pass on conv2 layer with conv1 activations as the input.

### Visualize the activations

In [ ]:
conv1_activations.shape

In [ ]:
np_act_conv1 = conv1_activations.detach().numpy()[0]

In [ ]:
np_act_conv1.shape

In [ ]:
for _, i in enumerate(np_act_conv1):
    plt.imshow(i, cmap='gray')
    plt.show()
    if _==5:
        break

### Activations of Convolution Layer 1

In [ ]:
conv1_activations = model.conv1.forward(number7.reshape(1,1,28,28))

In [ ]:
pooling = nn.MaxPool2d(2)

In [ ]:
c1_act_pool = pooling(conv1_activations)

In [ ]:
c1_act_pool.shape

### Activations of Convolution Layer 2

In [ ]:
conv2_activations = model.conv2.forward(c1_act_pool)

In [ ]:
conv2_activations.shape

In [ ]:
c2_act_pool = pooling(conv2_activations)

In [ ]:
c2_act_pool.shape

In [ ]:
for _, i in enumerate(conv2_activations.detach().numpy()[0]):
    plt.imshow(i, cmap='gray')
    plt.show()
    if _==5:
        break

### Activations of 1st Dense Layer

In [ ]:
fc1_out = model.fc1.forward(c2_act_pool.reshape(-1,))

In [ ]:
fc1_out

In [ ]:
relu = nn.ReLU()

In [ ]:
fc1_out_relu = relu(fc1_out)

### Activations of 2nd Dense Layer

In [ ]:
fc2_out = model.fc2.forward(fc1_out_relu)

In [ ]:
# this is your embeddings!
fc2_out